In [5]:
import warnings
warnings.filterwarnings("ignore")

import argparse
import logging
import os
import sys
import time

from collections import defaultdict, OrderedDict

import matplotlib
import numpy as np
import soundfile as sf
import torch
import yaml

from tensorboardX import SummaryWriter
from torch.utils.data import DataLoader
from tqdm import tqdm

import sys
sys.path.append("../")

from pyfiles.dataset import PretrainingMelDataset

import seq2seq_vc
import seq2seq_vc.models
import seq2seq_vc.losses
import seq2seq_vc.trainers
import seq2seq_vc.collaters

# from seq2seq_vc.datasets import ParallelVCMelDataset
from torch.utils.data import Dataset

from seq2seq_vc.utils import read_hdf5
from seq2seq_vc.utils.types import str_or_none

# set to avoid matplotlib error in CLI environment
import matplotlib

matplotlib.use("Agg")
import matplotlib.pyplot as plt

from seq2seq_vc.schedulers.warmup_lr import WarmupLR

scheduler_classes = dict(warmuplr=WarmupLR)

class Dict2Obj(object):
    def __init__(self, dictionary):
        """Constructor"""
        for key in dictionary:
            setattr(self, key, dictionary[key])

import joblib
import glob
datasplit = list(np.load("./data_split_ARCTIC.npy", allow_pickle=True))

In [6]:
# Dataset Variables
dataset_dir = "/mntcephfs/lee_dataset/tts/LibriTTS_R/"
# feat_base_dir = "/mntcephfs/lab_data/shoinoue/Dataset/LibriTTS_R/features/"
feat_base_dir = "/mntcephfs/data/audiow/shoinoue/Dataset/LibriTTS_R/features/"

scaler = {}
scaler_filename = f"ckpts/scalers/LibriTTS-R_16000.save"
scaler["mel"] = joblib.load(scaler_filename)
scaler_filename = f"ckpts/scalers/LibriTTS-R_80mel.save"
scaler["80mel"] = joblib.load(scaler_filename)
scaler_filename = f"ckpts/scalers/LibriTTS-R_hifiganmel.save"
scaler["hifiganmel"] = joblib.load(scaler_filename)
scaler_filename = f"ckpts/scalers/LibriTTS-R_wavlm.save"
scaler["wavlm"] = joblib.load(scaler_filename)

In [9]:
# checkpoint_path = "/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts_16000/pretraining_nocondition_LibriTTS-R_hubert80mel_norepeating_smaller/checkpoint-200000steps.pkl"
# checkpoint_path = "/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts_16000/pretraining_nocondition_LibriTTS-R_hubert80mel_norepeating_small/checkpoint-200000steps.pkl"
# checkpoint_path = "/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts_16000/pretraining_nocondition_LibriTTS-R_hubert80mel_norepeating/checkpoint-200000steps.pkl"

# checkpoint_path = "/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts_16000/pretraining_nocondition_LibriTTS-R_1000hubert80mel_norepeating_small/checkpoint-200000steps.pkl"
# checkpoint_path = "/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts_16000/pretraining_nocondition_LibriTTS-R_1000hubert80mel_norepeating/checkpoint-200000steps.pkl"

# checkpoint_path = "/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts_16000/pretraining_nocondition_LibriTTS-R_300hubert80mel_norepeating_small/checkpoint-200000steps.pkl"
# checkpoint_path = "/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts_16000/pretraining_nocondition_LibriTTS-R_300hubert80mel_norepeating/checkpoint-200000steps.pkl"

checkpoint_path = "/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts_16000/pretraining_nocondition_LibriTTS-R_huberthifiganmel_norepeating/checkpoint-160000steps.pkl"

input_type = "mel"
inputoutput = [input_type, "mel"] # mel to mimic wavlm
if "smaller" in checkpoint_path:
    size = "smaller"
elif "small" in checkpoint_path:
    size = "small"
else:
    size = ""
if "addition" in checkpoint_path:
    conditiontype = "add"
elif "concatenation" in checkpoint_path:
    conditiontype = "concat"
elif "nocondition" in checkpoint_path:
    conditiontype = "nocondition"
if "80mel" in checkpoint_path:
    output_type = "80mel"
    input_type = "80mel" if input_type=="mel" else input_type
elif "hifiganmel" in checkpoint_path:
    output_type = "hifiganmel"
    input_type = "hifiganmel" if input_type=="mel" else input_type
else:
    output_type = "mel"

args = {}
args["rank"] = 0
sizename = f"_{size}" if size else ""
# args["outdir"] = f"/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts_16000/encoder_fine-tuning_{conditiontype}_{''.join(inputoutput)}_from1000hubert80mel_norepeating{sizename}/"
args["outdir"] = f"/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts_16000/encoder_fine-tuning_{conditiontype}_{''.join(inputoutput)}_fromhuberthifiganmelmel_norepeating{sizename}/"
args["config_path"] = f"./../egs/l2-arctic/cascade/conf/{size}m2mvtn.{''.join(inputoutput)}.yaml"
args["init_checkpoint"] = checkpoint_path
args["resume"] = ""
args["distributed"] = False
args = Dict2Obj(args)

# load main config
with open(args.config_path) as f:
    config = yaml.load(f, Loader=yaml.Loader)
config.update(vars(args))

config["batch_size"] = 64
config["model_params"]["conditiontype"] = conditiontype
config["init-mods"] = ["decoder", "feat_out", "prob_out", "postnet"]
config["freeze-mods"] = ["decoder"]
config["train_max_steps"] = 100000
if input_type in ["80mel", "hifiganmel"]:
    config["model_params"]["idim"] = 80
if output_type in ["80mel", "hifiganmel"]:
    config["model_params"]["odim"] = 80

In [10]:
device = torch.device("cuda")
torch.backends.cudnn.benchmark = True
torch.cuda.set_device(args.rank)
if not os.path.exists(args.outdir):
    os.makedirs(args.outdir)
    
### Dataset Preparation ###
dataset = {
    "train": PretrainingMelDataset(feat_base_dir, dataset_dir, [scaler[input_type],scaler[output_type]], "train", input_output_type=[input_type, output_type], defiling_ratio=[0,0,1], noembedding=True),
    "dev": PretrainingMelDataset(feat_base_dir, dataset_dir, [scaler[input_type],scaler[output_type]], "valid", input_output_type=[input_type, output_type], defiling_ratio=[0,0,1], noembedding=True),
}

collater_class = getattr(
    seq2seq_vc.collaters,
    config.get("collater_type", "ARM2MVCCollater"),
)
collater = collater_class()

sampler = {"train": None, "dev": None}
data_loader = {
    "train": DataLoader(
        dataset=dataset["train"],
        shuffle=True,
        collate_fn=collater,
        batch_size=config["batch_size"],
        num_workers=config["num_workers"],
        sampler=sampler["train"],
        pin_memory=config["pin_memory"],
    ),
    "dev": DataLoader(
        dataset=dataset["dev"],
        shuffle=True,
        collate_fn=collater,
        batch_size=config["batch_size"],
        num_workers=config["num_workers"],
        sampler=sampler["dev"],
        pin_memory=config["pin_memory"],
    ),
}

### Model Preparation ###
model_class = getattr(
    seq2seq_vc.models,
    config.get("model_type", "M2MVTN"),
)
model = model_class(**config["model_params"]).to(device)

if config.get("criterions", None):
    criterion = {
        criterion_class: getattr(seq2seq_vc.losses, criterion_class)(
            **criterion_paramaters
        )
        for criterion_class, criterion_paramaters in config["criterions"].items()
    }
else:
    raise ValueError("Please specify criterions in the config file.")

### optimizers and schedulers ###
optimizer_class = getattr(
    torch.optim,
    # keep compatibility
    config.get("optimizer_type", "Adam"),
)
optimizer = optimizer_class(
    model.parameters(),
    **config["optimizer_params"],
)
scheduler_class = scheduler_classes.get(config.get("scheduler_type", "warmuplr"))
scheduler = scheduler_class(
    optimizer=optimizer,
    **config["scheduler_params"],
)

### define trainer ###
trainer_class = getattr(
    seq2seq_vc.trainers,
    config.get("trainer_type", "ARM2MVCTrainer"),
)
trainer = trainer_class(
    steps=0,
    epochs=0,
    data_loader=data_loader,
    sampler=sampler,
    model=model,
    vocoder=None,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    config=config,
    device=device,
)

# load pretrained parameters from checkpoint
if len(args.init_checkpoint) != 0:
    trainer.load_trained_modules(
        args.init_checkpoint, init_mods=config["init-mods"]
    )

# resume from checkpoint
if len(args.resume) != 0:
    trainer.load_checkpoint(args.resume)

# freeze modules if necessary
if config.get("freeze-mods", None) is not None:
    assert type(config["freeze-mods"]) is list
    trainer.freeze_modules(config["freeze-mods"])

In [11]:
try:
    trainer.run()
finally:
    trainer.save_checkpoint(
        os.path.join(config["outdir"], f"checkpoint-{trainer.steps}steps.pkl")
    )
    logging.info(f"Successfully saved checkpoint @ {trainer.steps}steps.")

[train]:   0%|          | 3/100000 [00:07<54:26:43,  1.96s/it] Traceback (most recent call last):
  File "/mntcephfs/lab_data/shoinoue/miniconda3/envs/cuhk/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/mntcephfs/lab_data/shoinoue/miniconda3/envs/cuhk/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/mntcephfs/lab_data/shoinoue/miniconda3/envs/cuhk/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/mntcephfs/lab_data/shoinoue/miniconda3/envs/cuhk/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/mntcephfs/lab_data/shoinoue/miniconda3/envs/cuhk/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/mntcephfs/lab_data/shoinoue/miniconda3/envs/cuhk/lib